# BigDataProgramming TermProject

## 엔카 크롤링

### Driver Load

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json

In [2]:
driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))

In [3]:
# 페이지 로딩 후 대기 시간 설정
wait_time = 0.5

### 150개 추출 바탕으로 csv 추출

In [4]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# 페이지 순회를 위한 범위 지정 (예: 1~15페이지)
page_start = 1
page_end = 15  # 원하는 페이지 수까지 변경 가능
wait_time = 1  # 페이지 로딩 대기 시간

# 전체 추출 번호를 저장할 리스트
all_numbers = set()

# 크롬 드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 1단계: 차량 번호 추출
for page in range(page_start, page_end + 1):
    try:
        # 페이지 URL 업데이트
        url = f"https://car.encar.com/list/car?page={page}"
        driver.get(url)
        
        # 페이지 로딩 대기
        time.sleep(wait_time)  # 필요에 따라 WebDriverWait으로 변경 가능
        
        # data-impression 값 추출
        index = 1
        while True:
            try:
                xpath = f"/html/body/div/div[1]/div[4]/div[1]/div[{index}]"
                element = driver.find_element(By.XPATH, xpath)
                data_impression = element.get_attribute("data-impression")
                if data_impression:
                    # 맨 앞 번호만 추출
                    first_number = data_impression.split("|")[0]
                    all_numbers.add(first_number)  # 번호 누적 저장
                index += 1
            except:
                # 현재 페이지의 매물 끝
                break
        
        print(f"페이지 {page} 크롤링 완료, 현재까지 추출된 번호 수: {len(all_numbers)}")
    except Exception as e:
        print(f"페이지 {page}에서 오류 발생: {e}")

print("모든 페이지에서 추출된 번호들:")
print(all_numbers)

# 2단계: 추출된 번호를 이용하여 보험 이력 저장
def fetch_insurance_history_to_csv(driver, vehicle_ids, output_file="insurance_history.csv"):
    """Fetch insurance history for given vehicle IDs and save to CSV."""
    fieldnames = [
        "Vehicle ID", "Car Number", "Number Change History", "Owner Change History",
        "Total Loss History", "Flood Damage History", "Theft History",
        "Insurance Claim (My Car)", "Insurance Claim (Other Car)"
    ]
    
    data = []

    for vehicle_id in vehicle_ids:
        url = f"https://fem.encar.com/cars/report/accident/{vehicle_id}"
        driver.get(url)
        
        # Default values for missing data
        insurance_data = {
            "Vehicle ID": vehicle_id,
            "Car Number": "N/A",
            "Number Change History": "N/A",
            "Owner Change History": "N/A",
            "Total Loss History": "N/A",
            "Flood Damage History": "N/A",
            "Theft History": "N/A",
            "Insurance Claim (My Car)": "N/A",
            "Insurance Claim (Other Car)": "N/A"
        }

        try:
            # Extracting Car Number
            insurance_data["Car Number"] = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/dl/dd[1]"))
            ).text or "N/A"

            # Extracting Number Change History
            insurance_data["Number Change History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[3]/span/span[1]"
            ).text or "N/A"

            # Extracting Owner Change History
            insurance_data["Owner Change History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[3]/span/span[2]"
            ).text or "N/A"

            # Extracting Total Loss History
            insurance_data["Total Loss History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[1]"
            ).text or "N/A"

            # Extracting Flood Damage History
            insurance_data["Flood Damage History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[2]"
            ).text or "N/A"

            # Extracting Theft History
            insurance_data["Theft History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[3]"
            ).text or "N/A"

            # Extracting Insurance Claim (My Car)
            insurance_data["Insurance Claim (My Car)"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[5]/span"
            ).text or "N/A"

            # Extracting Insurance Claim (Other Car)
            insurance_data["Insurance Claim (Other Car)"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[6]/span"
            ).text or "N/A"

        except Exception as e:
            print(f"Error fetching data for Vehicle ID {vehicle_id}: {e}")

        data.append(insurance_data)

    # Write data to CSV
    with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)
    
    print(f"Insurance history saved to {output_file}")


# Execute the pipeline
try:
    fetch_insurance_history_to_csv(driver, list(all_numbers))
finally:
    driver.quit()

페이지 1 크롤링 완료, 현재까지 추출된 번호 수: 10
페이지 2 크롤링 완료, 현재까지 추출된 번호 수: 20
페이지 3 크롤링 완료, 현재까지 추출된 번호 수: 30
페이지 4 크롤링 완료, 현재까지 추출된 번호 수: 40
페이지 5 크롤링 완료, 현재까지 추출된 번호 수: 50
페이지 6 크롤링 완료, 현재까지 추출된 번호 수: 60
페이지 7 크롤링 완료, 현재까지 추출된 번호 수: 70
페이지 8 크롤링 완료, 현재까지 추출된 번호 수: 80
페이지 9 크롤링 완료, 현재까지 추출된 번호 수: 89
페이지 10 크롤링 완료, 현재까지 추출된 번호 수: 99
페이지 11 크롤링 완료, 현재까지 추출된 번호 수: 109
페이지 12 크롤링 완료, 현재까지 추출된 번호 수: 119
페이지 13 크롤링 완료, 현재까지 추출된 번호 수: 129
페이지 14 크롤링 완료, 현재까지 추출된 번호 수: 139
페이지 15 크롤링 완료, 현재까지 추출된 번호 수: 149
모든 페이지에서 추출된 번호들:
{'38435573', '38283966', '38616597', '38220504', '38611122', '38388473', '38392543', '38330613', '38225617', '38583054', '38507922', '37914981', '38424973', '38181938', '38544747', '38377130', '38622568', '38305543', '37568721', '37262861', '38178919', '38283429', '37716200', '38355037', '38409690', '38301425', '38623553', '38620905', '38581088', '38079985', '37712100', '38474832', '37739965', '38546859', '38590435', '37452529', '38627103', '38568758', '38571030', '38578554', '37926138'

### csv파일 확인

In [5]:
import pandas as pd

df = pd.read_csv("insurance_history.csv")
print(df.head())

   Vehicle ID Car Number Number Change History Owner Change History  \
0    38435573    68두6888                    0회                   1회   
1    38283966   250거2326                    0회                   1회   
2    38616597   384루9044                    0회                   2회   
3    38220504    67서5575                    0회                   2회   
4    38611122   179서4344                    1회                   3회   

  Total Loss History Flood Damage History Theft History  \
0              전손 0회         침수(전손,분손) 0회         도난 0회   
1              전손 0회         침수(전손,분손) 0회         도난 0회   
2              전손 0회         침수(전손,분손) 0회         도난 0회   
3              전손 0회         침수(전손,분손) 0회         도난 0회   
4              전손 0회         침수(전손,분손) 0회         도난 0회   

  Insurance Claim (My Car) Insurance Claim (Other Car)  
0             2회1,596,600원               1회/2,816,620원  
1                       없음                          없음  
2             1회1,801,320원                     